
# ProteinDT (Hugging Face) — Download PDB & get 3Di

In [ ]:
#@title Mount Google Drive
USE_DRIVE = True
DRIVE_SUBDIR = "hf/proteinDT"
BASE_DIR = ""


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/LLM/Bioreasoner/testing_pipelines

from pathlib import Path
BASE_DIR = Path("")
#OUT_DIR  = BASE_DIR / "sft_test_demo"
print(f"Using Google Drive folder as BASE_DIR: {BASE_DIR}")


Mounted at /content/drive
/content/drive/MyDrive/LLM/Bioreasoner/testing_pipelines
Using Google Drive folder as BASE_DIR: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT


This commendline authorize the foldseek to execute. Make sure the foldseek binary file are in correct location.

In [35]:
!chmod +x bin/foldseek

In [2]:

#@title Install dependencies
!pip -q install --upgrade huggingface_hub datasets pandas biopython lmdb tqdm rich


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 60.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but 

In [3]:
from pathlib import Path
from huggingface_hub import snapshot_download
import os, json, lmdb, pickle, pandas as pd
from tqdm import tqdm
from rich import print as rprint

## Making PDB & 3Di tokens

This part of code will search through the chunk submitted to the openai request and build a worklist. It will then search on experimental mmCIFs (RCSB) or AF2 (PDB/CIF).




**Don't use mmCif files!!!! Process may fail when process mmCif using foldseek!! for unknown reason**

Will fix this in the future. But currently let's only use PDB files.




Technically this code will search for experimental Cif first, if files not found, then it will search on AF2(Predicted) dataset intead. However we currently cannot use CIF, so now we only use AF2.

Roughly 6 min/ 1k sequence, depends on internet connection.

In [ ]:
# === Download AlphaFold **PDB** for all UniProt in the first 10 submitted batches (10k) ===
import json, time, re
from pathlib import Path
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from tqdm import tqdm

# ---- paths (adjust BASE_DIR if needed) ----
BASE_DIR       = Path("")
OUT_DIR_FIXED  = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"
STRUCTURE_HOME = BASE_DIR / "downloads"
STRUCTURE_HOME.mkdir(parents=True, exist_ok=True)

AF2_PDB_LOG_CSV = OUT_DIR_FIXED / "structures_downloaded_af2_pdb.csv"

# AlphaFold PDB (not CIF)
AF2_PDB_URLS = [
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v4.pdb",
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v3.pdb",
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v2.pdb",
]

TIMEOUT = 25
PAUSE   = 0.02

def make_session():
    s = requests.Session()
    retries = Retry(
        total=4, backoff_factor=0.5,
        status_forcelist=[429,500,502,503,504],
        allowed_methods=["GET"]
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.mount("http://",  HTTPAdapter(max_retries=retries))
    return s

session = make_session()

def parse_chunk_idx(p: Path) -> int | None:
    m = re.search(r"_chunk(\d{3})\.json$", p.name)
    return int(m.group(1)) if m else None

def download_af2_pdb(uid: str, out_dir: Path) -> Path | None:
    """
    Try AF2 PDB v4 -> v3 -> v2 for the (canonical) UniProt accession.
    Writes immediately and returns saved Path or None.
    """
    out_dir.mkdir(parents=True, exist_ok=True)
    uid_base = uid.split("-")[0]  # handle isoforms like P12345-2
    for tpl in AF2_PDB_URLS:
        url = tpl.format(uid=uid_base)
        out = out_dir / Path(url).name
        try:
            r = session.get(url, timeout=TIMEOUT)
            if r.status_code == 200 and r.content and len(r.content) > 1024:
                out.write_bytes(r.content)
                return out
        except Exception:
            pass
        time.sleep(PAUSE)
    return None

# ---- discover the 10 submitted batches (chunks 000–009) ----
info_files = sorted(OUT_DIR_FIXED.glob("protein2desc_fixedans_batch_info_*_chunk???.json"))
# keep the first 10 unique chunk indices (000..009). If you want *all* submitted, drop the [:10].
chunk_indices = sorted({parse_chunk_idx(p) for p in info_files if parse_chunk_idx(p) is not None})[:10]
assert chunk_indices, "No submitted batch_info files found."

# collect matching meta files for those chunks
meta_files = []
for ci in chunk_indices:
    # any meta file with this chunk index
    meta_files += list(OUT_DIR_FIXED.glob(f"protein2desc_fixedans_meta_*_chunk{ci:03d}.json"))

assert meta_files, "No meta files found for the discovered chunks."

# ---- gather unique UniProt IDs from these meta files ----
uids = []
for p in meta_files:
    try:
        meta = json.loads(p.read_text(encoding="utf-8"))
        for _cid, rec in meta.items():
            uid = str(rec.get("uniprot_id") or "").strip()
            if uid:
                uids.append(uid)
    except Exception:
        pass

uids = list(dict.fromkeys(uids))  # unique, preserve order
print(f"Found UniProt IDs in {len(meta_files)} meta files (chunks {chunk_indices[0]}..{chunk_indices[-1]}): {len(uids)}")

# ---- resume-safe: load previous AF2 PDB log ----
if AF2_PDB_LOG_CSV.exists() and AF2_PDB_LOG_CSV.stat().st_size > 0:
    df_log_old = pd.read_csv(AF2_PDB_LOG_CSV)
    have_uids = set(df_log_old["uniprot_id"].astype(str)) if "uniprot_id" in df_log_old.columns else set()
else:
    df_log_old = pd.DataFrame(columns=["uniprot_id","pdb_id","file","source"])
    have_uids  = set()

# Skip already-downloaded
targets = [u for u in uids if u not in have_uids]
print(f"Already logged: {len(have_uids)}  |  To download now: {len(targets)}")

new_records = []
for uid in tqdm(targets, desc="AF2 PDB downloads"):
    out_dir = STRUCTURE_HOME / uid
    saved = download_af2_pdb(uid, out_dir)
    if saved and saved.exists() and saved.stat().st_size > 1024:
        new_records.append({
            "uniprot_id": uid,
            "pdb_id": Path(saved).stem,      # AF-<UID>-F1-model_vX
            "file": str(saved),
            "source": "AF2_PDB"
        })

# ---- write/merge log ----
df_new = pd.DataFrame(new_records, columns=["uniprot_id","pdb_id","file","source"])
df_out = pd.concat([df_log_old, df_new], ignore_index=True)
df_out = df_out.drop_duplicates(subset=["uniprot_id","pdb_id"]).reset_index(drop=True)
df_out.to_csv(AF2_PDB_LOG_CSV, index=False)

# ---- summary ----
n_total = len(uids)
n_have  = df_out["uniprot_id"].nunique() if "uniprot_id" in df_out.columns else 0
n_new   = len(new_records)

print("\nSummary:")
print(f"  Batches covered (chunks): {chunk_indices}")
print(f"  Total UniProt in these batches: {n_total}")
print(f"  Newly downloaded this run: {n_new}")
print(f"  Total AF2 PDB entries logged: {n_have}")
print("AF2 PDB log:", AF2_PDB_LOG_CSV)

# small preview
try:
    display(df_out.tail(5))
except:
    pass

Found UniProt IDs in 10 meta files (chunks 0..9): 10000
Already logged: 0  |  To download now: 10000


AF2 PDB downloads: 100%|██████████| 10000/10000 [1:08:22<00:00,  2.44it/s]


Summary:
  Batches covered (chunks): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  Total UniProt in these batches: 10000
  Newly downloaded this run: 7806
  Total AF2 PDB entries logged: 7806
AF2 PDB log: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/structures_downloaded_af2_pdb.csv


,uniprot_id,pdb_id,file,source
7801,C3KDX7,AF-C3KDX7-F1-model_v4,/content/drive/MyDrive/LLM/Bioreasoner/data/hf...,AF2_PDB
7802,Q817F2,AF-Q817F2-F1-model_v4,/content/drive/MyDrive/LLM/Bioreasoner/data/hf...,AF2_PDB
7803,A7HX46,AF-A7HX46-F1-model_v4,/content/drive/MyDrive/LLM/Bioreasoner/data/hf...,AF2_PDB
7804,Q6D222,AF-Q6D222-F1-model_v4,/content/drive/MyDrive/LLM/Bioreasoner/data/hf...,AF2_PDB
7805,P21305,AF-P21305-F1-model_v4,/content/drive/MyDrive/LLM/Bioreasoner/data/hf...,AF2_PDB


in this part it wil tranform all the downloaded pdb files to 3Di Tokens using foldseek.

Foldseek does not provide checkpoints, so the only two ways to use foldseek: 1) host local server (linux only, don't recommend if you are using wsl2, too many bugs) 2) use the binary files provided in protrek repo.

Also roughly 7 min/ 1k. Normally only 80% of sequence can get full data.

In [ ]:
# === Build 3Di (Foldseek) for first 10 submitted batches (PDB only) — SKIP if 3Di exists ===
import json, re, time
from pathlib import Path
from typing import Dict, Tuple, List
import pandas as pd
from tqdm import tqdm
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# -------------------- Paths (edit BASE_DIR if needed) --------------------
BASE_DIR        = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")
OUT_DIR_FIXED   = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"
STRUCTURE_HOME  = BASE_DIR / "downloads"
SFT_OUT_DIR     = BASE_DIR / "sft_build"
SFT_OUT_DIR.mkdir(parents=True, exist_ok=True)

THREEDI_DIR     = SFT_OUT_DIR / "foldseek_3di_pdb"
THREEDI_DIR.mkdir(parents=True, exist_ok=True)
MANIFEST_CSV    = THREEDI_DIR / "3di_manifest_pdb.csv"
FAILED_TXT      = THREEDI_DIR / "3di_failed_pdb.txt"

# Skip flag: if True, we won't recompute 3Di for UIDs that already have *.3di.txt
SKIP_IF_3DI_EXISTS = True

# -------------------- Foldseek util --------------------
from foldseek_util import get_struc_seq
try:
    FOLDSEEK_BIN
except NameError:
    FOLDSEEK_BIN = Path("bin/foldseek")  # e.g., Path("/usr/local/bin/foldseek"))

# -------------------- AF2 (PDB) download helpers --------------------
AF2_PDB_URLS = [
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v4.pdb",
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v3.pdb",
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v2.pdb",
]
TIMEOUT = 25
PAUSE   = 0.02

def make_session():
    s = requests.Session()
    retries = Retry(
        total=4, backoff_factor=0.5,
        status_forcelist=[429,500,502,503,504],
        allowed_methods=["GET"]
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.mount("http://",  HTTPAdapter(max_retries=retries))
    return s

session = make_session()
AF2_PDB_RE = re.compile(r"^AF-(?P<uid>[^-]+)-F1-model_v(?P<v>\d+)\.pdb$", re.I)

def find_best_af2_pdb(uid_dir: Path) -> Path | None:
    best, best_v = None, -1
    for p in uid_dir.glob("AF-*.pdb"):
        m = AF2_PDB_RE.match(p.name)
        if not m:
            continue
        v = int(m.group("v"))
        if v > best_v:
            best_v, best = v, p
    return best

def ensure_af2_pdb(uid: str) -> Path | None:
    d = STRUCTURE_HOME / uid
    d.mkdir(parents=True, exist_ok=True)
    best = find_best_af2_pdb(d)
    if best and best.exists() and best.stat().st_size > 1024:
        return best
    uid_base = uid.split("-")[0].strip()
    for tpl in AF2_PDB_URLS:
        url = tpl.format(uid=uid_base)
        out = d / Path(url).name
        try:
            r = session.get(url, timeout=TIMEOUT)
            if r.status_code == 200 and r.content and len(r.content) > 1024:
                out.write_bytes(r.content)
                return out
        except Exception:
            pass
        time.sleep(PAUSE)
    return None

# -------------------- Batch discovery (first 10 chunks) --------------------
def parse_chunk_idx(p: Path) -> int | None:
    m = re.search(r"_chunk(\d{3})\.json$", p.name)
    return int(m.group(1)) if m else None

info_files = sorted(OUT_DIR_FIXED.glob("protein2desc_fixedans_batch_info_*_chunk???.json"))
chunk_indices_all = sorted({parse_chunk_idx(p) for p in info_files if parse_chunk_idx(p) is not None})
assert chunk_indices_all, f"No submitted batch_info files found under {OUT_DIR_FIXED}"

chunk_indices = chunk_indices_all[:10]  # first 10 batches

meta_files = []
for ci in chunk_indices:
    meta_files += list(OUT_DIR_FIXED.glob(f"protein2desc_fixedans_meta_*_chunk{ci:03d}.json"))
assert meta_files, "No meta files found for discovered chunks."

uids = []
for p in meta_files:
    try:
        meta = json.loads(p.read_text(encoding="utf-8"))
        for _, rec in meta.items():
            uid = str(rec.get("uniprot_id") or "").strip()
            if uid: uids.append(uid)
    except Exception:
        pass
uids = list(dict.fromkeys(uids))
TOTAL_TARGETS = len(uids)
print(f"Targets from chunks {chunk_indices[0]}..{chunk_indices[-1]}: {TOTAL_TARGETS}")

# -------------------- 3Di build config --------------------
CHAIN_PROBE       = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
CHAIN_SEP_TOKEN   = "<|chain_sep|>"
ADD_CHAIN_ID_TAG  = True
CHAIN_ID_FMT      = "<|chain:{cid}|>"

def concat_chains(parsed: Dict[str, Tuple[str, str]]) -> Tuple[str, str, List[Dict]]:
    chain_ids = sorted(parsed.keys())
    aa_parts, di_parts, meta = [], [], []
    for cid in chain_ids:
        aa_seq, di_seq = parsed[cid][0], parsed[cid][1]
        if ADD_CHAIN_ID_TAG:
            aa_parts.append(CHAIN_ID_FMT.format(cid=cid))
            di_parts.append(CHAIN_ID_FMT.format(cid=cid))
        aa_parts.append(aa_seq)
        di_parts.append(di_seq.lower())
        meta.append({"chain": cid, "aa_len": len(aa_seq), "di_len": len(di_seq)})
    aa_concat = f" {CHAIN_SEP_TOKEN} ".join(aa_parts)
    di_concat = f" {CHAIN_SEP_TOKEN} ".join(di_parts)
    return aa_concat, di_concat, meta

def write_outputs(uid: str, pdb_file: Path, aa_concat: str, di_concat: str, chain_meta: List[Dict]):
    (THREEDI_DIR / f"{uid}.3di.txt").write_text(di_concat, encoding="utf-8")
    (THREEDI_DIR / f"{uid}.aa.txt").write_text(aa_concat, encoding="utf-8")
    meta = {
        "uniprot_id": uid,
        "pdb_file": str(pdb_file),
        "chain_sep_token": CHAIN_SEP_TOKEN,
        "add_chain_id_tag": ADD_CHAIN_ID_TAG,
        "chain_id_fmt": CHAIN_ID_FMT,
        "chains": chain_meta,
        "aa_total_len": sum(m["aa_len"] for m in chain_meta),
        "di_total_len": sum(m["di_len"] for m in chain_meta),
    }
    (THREEDI_DIR / f"{uid}.chains.json").write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")
    return meta

# Helper: check if a UID already has a 3Di file
def has_3di(uid: str) -> bool:
    f = THREEDI_DIR / f"{uid}.3di.txt"
    return f.exists() and f.stat().st_size > 0

# -------------------- Process: ensure PDB → 3Di (skip if exists) --------------------
rows = []
failed = []
have_pdb_count = 0
three_di_ok = 0
skipped_count = 0

# If a previous manifest exists, load it so we keep old rows
df_prev = pd.read_csv(MANIFEST_CSV) if MANIFEST_CSV.exists() and MANIFEST_CSV.stat().st_size > 0 else pd.DataFrame()

for uid in tqdm(uids, desc="Ensure AF2 PDB & build 3Di"):
    # If we already have a 3Di file and skipping is enabled, move on
    if SKIP_IF_3DI_EXISTS and has_3di(uid):
        skipped_count += 1
        continue

    # 1) ensure PDB
    pdb_path = ensure_af2_pdb(uid)
    if pdb_path and pdb_path.exists() and pdb_path.stat().st_size > 1024:
        have_pdb_count += 1
    else:
        failed.append(f"{uid}\tno_af2_pdb")
        continue

    # 2) get all chains → 3Di
    try:
        parsed = get_struc_seq(str(FOLDSEEK_BIN), str(pdb_path), CHAIN_PROBE, plddt_mask=True)
        if not parsed:
            failed.append(f"{uid}\tno_chains_returned")
            print(f"No chains returned for {uid}")
            continue

        aa_concat, di_concat, chain_meta = concat_chains(parsed)
        meta = write_outputs(uid, pdb_path, aa_concat, di_concat, chain_meta)
        three_di_ok += 1

        rows.append({
            "uniprot_id": uid,
            "pdb_file": str(pdb_path),
            "aa_total_len": meta["aa_total_len"],
            "di_total_len": meta["di_total_len"],
            "n_chains": len(meta["chains"]),
            "di_path": str(THREEDI_DIR / f"{uid}.3di.txt"),
            "aa_path": str(THREEDI_DIR / f"{uid}.aa.txt"),
            "meta_path": str(THREEDI_DIR / f"{uid}.chains.json"),
        })
    except Exception as e:
        failed.append(f"{uid}\t{type(e).__name__}:{e}")

# -------------------- Save manifest & failed list --------------------
df_new = pd.DataFrame(rows, columns=["uniprot_id","pdb_file","aa_total_len","di_total_len","n_chains","di_path","aa_path","meta_path"])
if not df_prev.empty:
    df_all = pd.concat([df_prev, df_new], ignore_index=True)
    df_all = df_all.drop_duplicates(subset=["uniprot_id"], keep="last").reset_index(drop=True)
else:
    df_all = df_new
df_all.to_csv(MANIFEST_CSV, index=False)
Path(FAILED_TXT).write_text("\n".join(sorted(set(failed))), encoding="utf-8")

# -------------------- Stats --------------------
print("\n=== STATS (10k batch set) ===")
print(f"Total targets (UniProt):           {TOTAL_TARGETS}")
print(f"Skipped (already had 3Di):         {skipped_count}")
print(f"With AF2 PDB available (new run):  {have_pdb_count}")
print(f"3Di successfully generated (new):  {three_di_ok}")
print(f"Failures (see file):               {len(set(failed))}  -> {FAILED_TXT.name}")
print("Manifest written to:               ", MANIFEST_CSV)

# small peek
try:
    display(df_all.tail(5))
except:
    pass


Targets from chunks 0..9: 10000


Ensure AF2 PDB & build 3Di:  18%|█▊        | 1843/10000 [07:24<46:08,  2.95it/s]

Targets from chunks 0..9: 10000

Ensure AF2 PDB & build 3Di:  25%|██▍       | 2473/10000 [16:25<49:58,  2.51it/s]

In [ ]:
#@title Progress Checker
from pathlib import Path
import pandas as pd
from datetime import datetime

BASE_DIR      = Path("")
THREEDI_DIR   = BASE_DIR / "sft_build" / "foldseek_3di_pdb"
STRUCT_HOME   = BASE_DIR / "downloads"

PROGRESS_LOG  = THREEDI_DIR / "3di_progress_log.csv"
MANIFEST_CSV  = THREEDI_DIR / "3di_manifest_pdb.csv"

# 1) how many PDBs exist on disk so far?
pdb_files = list(STRUCT_HOME.rglob("AF-*-F1-model_v*.pdb"))
print("AF2 PDB files on disk:", len(pdb_files))

# 2) how many 3Di outputs exist?
three_di_files = list(THREEDI_DIR.glob("*.3di.txt"))
print("3Di outputs on disk:", len(three_di_files))

# 3) tail the progress log
if PROGRESS_LOG.exists():
    dfp = pd.read_csv(PROGRESS_LOG)
    print("\nLog rows:", len(dfp))
    print(dfp.tail(10).to_string(index=False))
else:
    print("\nNo progress log yet:", PROGRESS_LOG)

# 4) peek the manifest
if MANIFEST_CSV.exists():
    dfm = pd.read_csv(MANIFEST_CSV)
    print("\nManifest rows:", len(dfm))
    print(dfm.tail(5))
else:
    print("\nNo manifest yet:", MANIFEST_CSV)

AF2 PDB files on disk: 7807
3Di outputs on disk: 1902

No progress log yet: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/sft_build/foldseek_3di_pdb/3di_progress_log.csv

No manifest yet: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/sft_build/foldseek_3di_pdb/3di_manifest_pdb.csv


# Debug attempt

ignore the code below if you just want to run the pipeline

In [ ]:
# === Rebuild worklist_5k.csv from your existing outputs ===
from pathlib import Path
import json, re
import pandas as pd

BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")  # adjust if needed
# Put the worklist in the same OUT_DIR you’re using for the structure pipeline
OUT_DIR  = BASE_DIR / "gpt_batch_protein2desc"   # or "sft_build" if that’s what you used
OUT_DIR.mkdir(parents=True, exist_ok=True)

WORKLIST_CSV = OUT_DIR / "worklist_5k.csv"

def gather_from_batch_meta(dirpath: Path):
    rows = []
    for p in sorted(dirpath.glob("protein2desc_*meta*.json")):
        try:
            meta = json.loads(p.read_text(encoding="utf-8"))
            for cid, m in meta.items():
                uid = str(m.get("uniprot_id") or "").strip()
                seq = str(m.get("sequence") or m.get("protein_sequence") or "").strip()
                if uid:
                    rows.append((uid, seq))
        except Exception:
            pass
    return rows

def gather_from_sft(dirpath: Path):
    rows = []
    for p in sorted(dirpath.glob("protein2desc_*sft*.json")):
        try:
            data = json.loads(p.read_text(encoding="utf-8"))
            if isinstance(data, list):
                for r in data:
                    inp = r.get("input") or {}
                    uid = str(inp.get("uniprot_id") or "").strip()
                    seq = str(inp.get("protein_sequence") or "").strip()
                    if uid:
                        rows.append((uid, seq))
        except Exception:
            pass
    return rows

rows = []
# 1) batch meta JSONs created during submissions/fetch
rows += gather_from_batch_meta(OUT_DIR)
# 2) any SFT JSONs you built (parsed/raw)
rows += gather_from_sft(OUT_DIR)

# de-dup, prefer the first non-empty sequence we saw for a given UniProt
uniq = {}
for uid, seq in rows:
    if uid not in uniq or (not uniq[uid] and seq):
        uniq[uid] = seq

df = pd.DataFrame(
    [{"uniprot_id": uid, "protein_sequence": uniq[uid]} for uid in uniq.keys()]
).sort_values("uniprot_id").reset_index(drop=True)

# If you only want the first 5k, slice here:
df_5k = df.head(5000).copy()

df_5k.to_csv(WORKLIST_CSV, index=False)
print(f"Saved worklist: {WORKLIST_CSV}  rows={len(df_5k)}")
df_5k.head(5)

In [12]:
import os, json, re, html, random
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from tqdm import tqdm

# ---------- CONFIG ----------
BASE_DIR        = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")  # Change if needed
RESULTS_ROOT    = BASE_DIR / "gpt_batch_protein2desc_fixed_answer" / "sft_results"       # where 5k come from
STRUCTURE_HOME  = BASE_DIR / "downloads"                                            # per-UniProt structure dir
FOLDSEEK_BIN    = Path("/content/drive/MyDrive/LLM/Bioreasoner/testing_pipelines/bin/foldseek")                                              # local foldseek binary
OUT_DIR         = BASE_DIR / "sft_build"
OUT_DIR.mkdir(parents=True, exist_ok=True)

N_LIMIT         = 5000         # take 5k
USE_GT_RESPONSE = True         # True: use ground-truth description from meta; False: parse <answer> from output
SHUFFLE_SEED    = 7

OUT_JSONL       = OUT_DIR / "protein2desc_seq+3di.jsonl"
LOG_BAD         = OUT_DIR / "protein2desc_missing_structure.log"

# Prompt template used in "prompt" (SFT input). Keep it natural and short.
PROMPT_TEMPLATE = (
    "You are a professional protein biologist. Based on the amino-acid sequence (and structure if available), "
    "write a concise, biologically accurate 2–4 sentence description of the protein."
)

In [13]:
try:
    # Provided by SaProt/ProTrek repos
    from foldseek_util import get_struc_seq
except Exception as e:
    raise ImportError(
        "Could not import utils.foldseek_util.get_struc_seq. "
        "Please ensure you have the SaProt/ProTrek utilities on your PYTHONPATH.\n"
        "Repo example (SaProt): utils/foldseek_util.py has get_struc_seq()."
    )

In [14]:
# ---------- Helpers ----------
def strip_html(s: str) -> str:
    s = s or ""
    s = html.unescape(s)
    return re.sub(r"<[^>]+>", " ", s).replace("\xa0"," ").strip()

def _extract_tag_loose(text: str, tag: str):
    if not text:
        return "", 0, False
    open_pat  = re.compile(fr"<{tag}\b[^>]*>", re.I)
    close_pat = re.compile(fr"</{tag}\s*>", re.I)
    m_open = open_pat.search(text)
    if not m_open:
        return "", 0, False
    m_close = close_pat.search(text, m_open.end())
    if m_close:
        content = text[m_open.end():m_close.start()]
        endpos  = m_close.end()
    else:
        content = text[m_open.end():]
        endpos  = len(text)
    content = content.strip().rstrip('", \t\r\n')
    return content, endpos, True

def parse_thinking_answer_loose(text: str):
    if not text:
        return "", ""
    thinking, th_end, th_found = _extract_tag_loose(text, "thinking")
    remainder = text[th_end:] if th_found else text
    answer, _, an_found = _extract_tag_loose(remainder, "answer")
    if not th_found and an_found:
        m_open_ans = re.search(r"<answer\b[^>]*>", remainder, re.I)
        cutoff = m_open_ans.start() if m_open_ans else len(text)
        pre = text[:cutoff].strip().rstrip('", \t\r\n')
        thinking = pre if pre else thinking
    if not th_found and not an_found:
        thinking = text.strip().rstrip('", \t\r\n')
        answer   = ""
    return thinking, answer

def two_line_pairs(path: Path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line: break
            val_line = f.readline()
            if not val_line: break
            yield id_line.strip(), val_line.strip()

def load_swissprotclap(base_dir: Path):
    sp = base_dir / "SwissProtCLAP"
    seq_fp, txt_fp = sp / "protein_sequence.txt", sp / "text_sequence.txt"
    id2seq = {pid: seq for pid, seq in two_line_pairs(seq_fp)}
    id2cap = {pid: strip_html(txt) for pid, txt in two_line_pairs(txt_fp)}
    return id2seq, id2cap


### Downloading Cif/PDB

In [25]:
STRUCTURE_HOME

PosixPath('/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/downloads')

In [30]:
# === Fallback downloader: AlphaFold mmCIF by UniProt for missing entries ===
import json, time
from pathlib import Path
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from tqdm import tqdm

# Reuse your existing paths/vars
# BASE_DIR, OUT_DIR, STRUCTURE_HOME already defined above
WORK_CSV        = OUT_DIR / "worklist_5k.csv"
STRUCT_LOG_CSV  = OUT_DIR / "structures_downloaded.csv"
AF2_LOG_CSV     = OUT_DIR / "structures_downloaded_af2.csv"
AF2_MISSED_TXT  = OUT_DIR / "structures_missing_af2.txt"

# AlphaFold file name templates (try in this order)
AF2_CANDIDATES  = [
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v4.cif",
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v3.cif",
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v2.cif",
]

TIMEOUT = 25
PAUSE   = 0.02

def ensure_dir(p: Path): p.mkdir(parents=True, exist_ok=True)

def make_session():
    s = requests.Session()
    retries = Retry(
        total=4, backoff_factor=0.5,
        status_forcelist=[429,500,502,503,504],
        allowed_methods=["GET"]
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.mount("http://",  HTTPAdapter(max_retries=retries))
    return s

session = make_session()

def has_experimental_locally(uid: str) -> bool:
    d = STRUCTURE_HOME / uid
    if not d.exists(): return False
    return any(p.suffix.lower()==".cif" and not p.name.startswith("AF-") for p in d.glob("*.cif"))

def has_af2_locally(uid: str) -> bool:
    d = STRUCTURE_HOME / uid
    if not d.exists(): return False
    return any(p.suffix.lower()==".cif" and p.name.startswith("AF-") for p in d.glob("*.cif"))

def download_af2_cif(uniprot_id: str, out_dir: Path) -> Path | None:
    """
    Try AF2 v4 -> v3 -> v2 for a UniProt (canonical; handle isoforms by stripping '-2').
    Returns the saved Path or None.
    """
    uid_base = uniprot_id.split("-")[0]  # handle isoforms like P12345-2
    ensure_dir(out_dir)
    for url_tpl in AF2_CANDIDATES:
        url = url_tpl.format(uid=uid_base)
        fname = Path(url).name
        out = out_dir / fname
        try:
            r = session.get(url, timeout=TIMEOUT)
            if r.status_code == 200 and r.content and len(r.content) > 1024:
                out.write_bytes(r.content)
                return out
        except Exception:
            pass
        time.sleep(PAUSE)
    return None

# ---- Load the worklist and figure out who is missing any CIFs
df = pd.read_csv(WORK_CSV)
uids = df["uniprot_id"].astype(str).tolist()

# Experimental log (if exists)
exp_log = pd.read_csv(STRUCT_LOG_CSV) if STRUCT_LOG_CSV.exists() else pd.DataFrame(columns=["uniprot_id","pdb_id","file","chains"])
have_exp = set(exp_log["uniprot_id"].astype(str)) if len(exp_log) else set()

# Already-downloaded AF2 log (resume-safe)
af2_log = pd.read_csv(AF2_LOG_CSV) if AF2_LOG_CSV.exists() else pd.DataFrame(columns=["uniprot_id","pdb_id","file","chains","source"])
have_af2 = set(af2_log["uniprot_id"].astype(str)) if len(af2_log) else set()

to_fetch = [u for u in uids if u not in have_exp and u not in have_af2 and not has_af2_locally(u)]
print(f"Need AF2 for (no experimental + not already AF2): {len(to_fetch)}")

af2_records = []
af2_missed  = []

for uid in tqdm(to_fetch, desc="AF2 CIF download"):
    out_dir = STRUCTURE_HOME / uid
    saved = download_af2_cif(uid, out_dir)
    if saved and saved.exists() and saved.stat().st_size > 1024:
        af2_records.append({
            "uniprot_id": uid,
            "pdb_id": Path(saved).stem,     # 'AF-<UID>-F1-model_vX'
            "file": str(saved),
            "chains": "A",                  # AF2 is single chain
            "source": "AF2"
        })
    else:
        af2_missed.append(uid)

# Append/Save AF2 log (dedupe; resume-safe)
if af2_records:
    df_new = pd.DataFrame(af2_records)
    if AF2_LOG_CSV.exists():
        old = pd.read_csv(AF2_LOG_CSV)
        df_out = pd.concat([old, df_new], ignore_index=True)
    else:
        df_out = df_new
    df_out = df_out.drop_duplicates(subset=["uniprot_id","pdb_id"]).reset_index(drop=True)
    df_out.to_csv(AF2_LOG_CSV, index=False)

# Write missed list
Path(AF2_MISSED_TXT).write_text("\n".join(sorted(set(af2_missed))), encoding="utf-8")

print("\nAF2 summary:")
print("  New AF2 files:", len(af2_records))
print("  Total AF2 log rows:", (pd.read_csv(AF2_LOG_CSV).shape[0] if AF2_LOG_CSV.exists() else 0))
print("  Missed AF2:", len(af2_missed))
print("AF2 log:", AF2_LOG_CSV)
print("AF2 missed list:", AF2_MISSED_TXT)

# small preview
if AF2_LOG_CSV.exists():
    display(pd.read_csv(AF2_LOG_CSV).head(5))

Need AF2 for (no experimental + not already AF2): 4965


AF2 CIF download: 100%|██████████| 4965/4965 [46:26<00:00,  1.78it/s]


AF2 summary:
  New AF2 files: 3863
  Total AF2 log rows: 3863
  Missed AF2: 1102
AF2 log: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/sft_build/structures_downloaded_af2.csv
AF2 missed list: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/sft_build/structures_missing_af2.txt


,uniprot_id,pdb_id,file,chains,source
0,A1K1S0,AF-A1K1S0-F1-model_v4,/content/drive/MyDrive/LLM/Bioreasoner/data/hf...,A,AF2
1,A3CXH4,AF-A3CXH4-F1-model_v4,/content/drive/MyDrive/LLM/Bioreasoner/data/hf...,A,AF2
2,P52409,AF-P52409-F1-model_v4,/content/drive/MyDrive/LLM/Bioreasoner/data/hf...,A,AF2
3,Q1CT83,AF-Q1CT83-F1-model_v4,/content/drive/MyDrive/LLM/Bioreasoner/data/hf...,A,AF2
4,Q2JL77,AF-Q2JL77-F1-model_v4,/content/drive/MyDrive/LLM/Bioreasoner/data/hf...,A,AF2


#### Error Checking

In [17]:
from pathlib import Path
import pandas as pd

print("BASE_DIR      :", BASE_DIR)
print("STRUCTURE_HOME:", STRUCTURE_HOME)
print("Exists BASE_DIR?      ", BASE_DIR.exists())
print("Exists STRUCTURE_HOME?", STRUCTURE_HOME.exists())

WORK_CSV = OUT_DIR / "worklist_5k.csv"
df = pd.read_csv(WORK_CSV)

def has_exp(uid: str) -> bool:
    d = STRUCTURE_HOME / uid
    return d.exists() and any(p.suffix.lower()==".cif" and not p.name.startswith("AF-") for p in d.glob("*.cif"))

df["has_exp"] = df["uniprot_id"].astype(str).apply(has_exp)
print("Targets:", len(df))
print("Have experimental mmCIF already:", int(df["has_exp"].sum()))
print("Still missing experimental mmCIF:", int((~df["has_exp"]).sum()))

BASE_DIR      : /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT
STRUCTURE_HOME: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/downloads
Exists BASE_DIR?       True
Exists STRUCTURE_HOME? True
Targets: 4967
Have experimental mmCIF already: 0
Still missing experimental mmCIF: 4967


In [18]:
import requests, time

PDBe_MAP_URL = "https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/{uid}"
TIMEOUT = 25

def pdbe_map_uniprot(uid: str) -> dict:
    try:
        r = requests.get(PDBe_MAP_URL.format(uid=uid), timeout=TIMEOUT)
        if r.status_code != 200:
            return {}
        payload = r.json()
        out = {}
        if isinstance(payload, dict) and uid in payload:
            pdbs = payload[uid].get("PDB") or {}
            for pdb_id, lst in pdbs.items():
                chains = sorted({ item.get("chain_id") for item in lst if item.get("chain_id") })
                if chains:
                    out[pdb_id.lower()] = chains
        return out
    except Exception:
        return {}

sample = df.loc[~df["has_exp"], "uniprot_id"].astype(str).head(20).tolist()
for uid in sample:
    m = pdbe_map_uniprot(uid)
    print(uid, "→", f"{len(m)} PDBs" if m else "no mapping")
    time.sleep(0.05)

A1K1S0 → no mapping
A3CXH4 → no mapping
Q9P7I4 → no mapping
P52409 → no mapping
Q1CT83 → no mapping
Q2JL77 → no mapping
Q8KCG7 → no mapping
Q39IW5 → no mapping
Q9LZX8 → no mapping
D6W1X9 → no mapping
D6VX92 → no mapping
O86788 → no mapping
Q9CPJ2 → no mapping
C1CN25 → no mapping
A1AE55 → no mapping
B7LLT6 → no mapping
B7GW85 → no mapping
I7GFL2 → no mapping
G0KA64 → no mapping
Q2II86 → no mapping


In [19]:
import requests, time, re
from pathlib import Path

TEST_UID = "P04637"  # replace with any UniProt from your 5k list if you prefer
STRUCTURE_HOME.mkdir(parents=True, exist_ok=True)

def ensure_dir(p: Path): p.mkdir(parents=True, exist_ok=True)

def pdbe_map(uid):
    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/{uid}"
    r = requests.get(url, timeout=25)
    if r.status_code != 200:
        return {}
    payload = r.json()
    out = {}
    if isinstance(payload, dict) and uid in payload:
        pdbs = payload[uid].get("PDB") or {}
        for pdb_id, lst in pdbs.items():
            chains = sorted({ item.get("chain_id") for item in lst if item.get("chain_id") })
            if chains:
                out[pdb_id.lower()] = chains
    return out

def download_cif(pdb_id, out_path):
    url = f"https://files.rcsb.org/download/{pdb_id.upper()}.cif"
    r = requests.get(url, timeout=25)
    if r.status_code == 200 and len(r.content) > 1024:
        out_path.write_bytes(r.content)
        return True
    return False

m = pdbe_map(TEST_UID)
print("PDBs for", TEST_UID, ":", list(m.keys())[:5])

if m:
    uid_dir = STRUCTURE_HOME / TEST_UID
    ensure_dir(uid_dir)
    pdb_id = sorted(m.keys())[0]
    out = uid_dir / f"{pdb_id}.cif"
    ok = download_cif(pdb_id, out)
    print("Saved:", out, "OK:", ok, "size(MB):", out.stat().st_size/1024/1024 if out.exists() else 0)
else:
    print("No mapping for", TEST_UID)

PDBs for P04637 : []
No mapping for P04637


### mmCIF/PDB to Foldseek 3Di

In [37]:
# ===== Patched Cell 3 (robust): Convert mmCIF to Foldseek 3Di (experimental + AF2) =====
import os
from pathlib import Path
import pandas as pd
from tqdm import tqdm

# foldseek util from your repo
from foldseek_util import get_struc_seq

STRUCT_LOG_CSV    = OUT_DIR / "structures_downloaded.csv"      # experimental (if any)
AF2_LOG_CSV       = OUT_DIR / "structures_downloaded_af2.csv"  # AF2 fallback
PER_CHAIN_CSV     = OUT_DIR / "foldseek_3di_per_chain.csv"
CACHE_DIR         = OUT_DIR / "3di_cache"
ERR_LOG           = OUT_DIR / "foldseek_3di_errors.log"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

assert FOLDSEEK_BIN.exists() and os.access(FOLDSEEK_BIN, os.X_OK), f"Foldseek binary not executable at {FOLDSEEK_BIN}"

# Read both logs and unify
dfs = []
if STRUCT_LOG_CSV.exists():
    dfs.append(pd.read_csv(STRUCT_LOG_CSV))
if AF2_LOG_CSV.exists():
    dfs.append(pd.read_csv(AF2_LOG_CSV))
if dfs:
    df_structs = pd.concat(dfs, ignore_index=True).drop_duplicates(subset=["uniprot_id","pdb_id"])
else:
    df_structs = pd.DataFrame(columns=["uniprot_id","pdb_id","file","chains","source"])

def is_af2_row(row):
    # prefer explicit 'source' column if present
    s = row.get("source") if isinstance(row, dict) else (row["source"] if "source" in row else "")
    if isinstance(s, str) and s.strip().upper() == "AF2":
        return True
    # fallback: filename starts with AF-
    fname = Path(str(row["file"])).name
    return fname.startswith("AF-")

rows = []
errors = []
n_ok, n_skip, n_fail = 0, 0, 0

for _, r in tqdm(df_structs.iterrows(), total=len(df_structs), desc="Foldseek 3Di"):
    uid   = str(r["uniprot_id"])
    pdbid = str(r["pdb_id"]).lower()
    path  = Path(r["file"])

    if not path.exists() or path.stat().st_size < 1024:
        n_skip += 1
        continue

    af2 = is_af2_row(r)
    # chains: prefer provided list; otherwise AF2 -> ["A"]; else fallback to ["A"]
    raw_ch = str(r.get("chains", "")) if "chains" in r else ""
    chains = [c for c in raw_ch.split(",") if c] or ["A"]

    for ch in chains:
        cache_txt = CACHE_DIR / f"{uid}_{pdbid}_{ch}.3di.txt"
        if cache_txt.exists():
            fs = cache_txt.read_text(encoding="utf-8").strip()
            if fs:
                rows.append({
                    "uniprot_id": uid,
                    "pdb_id": pdbid,
                    "chain": ch,
                    "stru_str": fs,
                    "aa_len": None,
                    "stru_len": len(fs),
                    "source_file": str(path),
                    "source_kind": ("AF2" if af2 else "EXP")
                })
                n_ok += 1
            else:
                n_fail += 1
            continue

        try:
            # AF2: enable pLDDT masking; Experimental: no mask
            parsed = get_struc_seq(str(FOLDSEEK_BIN), str(path), [ch], plddt_mask=af2)
            if ch in parsed and parsed[ch] and len(parsed[ch]) >= 2:
                aa_seq = parsed[ch][0] or ""
                fs3di  = (parsed[ch][1] or "").lower()
                if fs3di:
                    cache_txt.write_text(fs3di, encoding="utf-8")
                    rows.append({
                        "uniprot_id": uid,
                        "pdb_id": pdbid,
                        "chain": ch,
                        "stru_str": fs3di,
                        "aa_len": len(aa_seq),
                        "stru_len": len(fs3di),
                        "source_file": str(path),
                        "source_kind": ("AF2" if af2 else "EXP")
                    })
                    n_ok += 1
                else:
                    n_fail += 1
                    errors.append(f"{uid}\t{pdbid}\t{ch}\tempty_3di")
            else:
                n_fail += 1
                errors.append(f"{uid}\t{pdbid}\t{ch}\tno_parsed_chain")
        except Exception as e:
            n_fail += 1
            errors.append(f"{uid}\t{pdbid}\t{ch}\t{type(e).__name__}: {e}")

# Build DataFrame with explicit columns (so it never has 0 columns)
cols = ["uniprot_id","pdb_id","chain","stru_str","aa_len","stru_len","source_file","source_kind"]
df_3di = pd.DataFrame(rows, columns=cols).drop_duplicates(subset=["uniprot_id","pdb_id","chain"])
df_3di.to_csv(PER_CHAIN_CSV, index=False)

# Error log (optional)
if errors:
    ERR_LOG.write_text("\n".join(errors), encoding="utf-8")

proteins = df_3di["uniprot_id"].nunique() if "uniprot_id" in df_3di.columns else 0
print(f"Saved per-chain 3Di to: {PER_CHAIN_CSV}  (rows={len(df_3di)}, proteins={proteins})")
print(f"Success: {n_ok}  |  skipped(missing/too small): {n_skip}  |  failed(parse/errors): {n_fail}")
if errors:
    print(f"Errors logged to: {ERR_LOG}")

# quick peek
df_3di.head(5)

Foldseek 3Di: 100%|██████████| 3866/3866 [07:06<00:00,  9.06it/s]

Saved per-chain 3Di to: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/sft_build/foldseek_3di_per_chain.csv  (rows=0, proteins=0)
Success: 0  |  skipped(missing/too small): 0  |  failed(parse/errors): 3866
Errors logged to: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/sft_build/foldseek_3di_errors.log


,uniprot_id,pdb_id,chain,stru_str,aa_len,stru_len,source_file,source_kind


In [36]:
import subprocess, os
print("Binary exists & exec?", os.path.exists(str(FOLDSEEK_BIN)), os.access(str(FOLDSEEK_BIN), os.X_OK))
print(subprocess.run([str(FOLDSEEK_BIN), "-v"], capture_output=True, text=True).stdout or "no -v")
print(subprocess.run([str(FOLDSEEK_BIN), "-h"], capture_output=True, text=True).stdout[:2000])

Binary exists & exec? True True
Foldseek enables fast and sensitive comparisons of large structure sets. It reaches sensitivities similar to state-of-the-art structural aligners while being at least 20,000 times faster.

Please cite:
van Kempen, M., Kim, S.S., Tumescheit, C., Mirdita, M., Lee, J., Gilchrist, C.L.M., Söding, J., and Steinegger, M. Fast and accurate protein structure search with Foldseek. Nature Biotechnology, doi:10.1038/s41587-023-01773-0 (2023)

foldseek Version: ef4e960ab84fc502665eb7b84573dfff9c2aa89d
© Michel van Kempen, Stephanie Kim, Charlotte Tumescheit, Milot Mirdita, Jeongjae Lee, Cameron L. M. Gilchrist, Johannes Söding, Martin Steinegger

usage: foldseek <command> [<args>]

Easy workflows for plain text input/output
  easy-search       	Structual search
  easy-cluster      	Slower, sensitive clustering
  easy-rbh          	Find reciprocal best hit
  easy-complexsearch	Complex level search

Main workflows for database input/output
  createdb          	Convert

In [39]:
import pandas as pd
from pathlib import Path
from foldseek_util import get_struc_seq

AF2_LOG_CSV = OUT_DIR / "structures_downloaded_af2.csv"
df_af2 = pd.read_csv(AF2_LOG_CSV)
row = df_af2.iloc[0]  # pick one
path = Path(row["file"])
uid  = row["uniprot_id"]

print("Testing:", uid, "| file:", path.name, "| exists:", path.exists(), "| size(MB):", round(path.stat().st_size/1024/1024, 2))
parsed = get_struc_seq(str(FOLDSEEK_BIN), str(path), ["A"], plddt_mask=True)
print("Chains returned:", list(parsed.keys()))
print("AA len, 3Di len:", len(parsed["A"][0]), len(parsed["A"][1]))
print("3Di head:", parsed["A"][1][:60])

Testing: A1K1S0 | file: AF-A1K1S0-F1-model_v4.cif | exists: True | size(MB): 0.43


ValueError: invalid literal for int() with base 10: 'MET'

In [ ]:
# --- Test get_struc_seq using an AlphaFold **PDB** file (chain A) ---
import pandas as pd, requests, time
from pathlib import Path
from foldseek_util import get_struc_seq

# Paths (adjust BASE_DIR if needed)
BASE_DIR       = Path("")
OUT_DIR        = BASE_DIR / "sft_build"
STRUCTURE_HOME = BASE_DIR / "downloads"
AF2_LOG_CSV    = OUT_DIR / "structures_downloaded_af2.csv"

# Foldseek binary location
try:
    FOLDSEEK_BIN
except NameError:
    FOLDSEEK_BIN = Path("bin/foldseek")  # e.g., Path("/usr/local/bin/foldseek")

AF2_PDB_URLS = [
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v4.pdb",
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v3.pdb",
    "https://alphafold.ebi.ac.uk/files/AF-{uid}-F1-model_v2.pdb",
]

def dl_af2_pdb(uid: str, out_dir: Path) -> Path | None:
    out_dir.mkdir(parents=True, exist_ok=True)
    uid_base = uid.split("-")[0]
    for tpl in AF2_PDB_URLS:
        url = tpl.format(uid=uid_base)
        p = out_dir / Path(url).name
        r = requests.get(url, timeout=25)
        if r.status_code == 200 and len(r.content) > 1024:
            p.write_bytes(r.content)
            return p
        time.sleep(0.05)
    return None

# Pick one AF2 entry you already logged
df_af2 = pd.read_csv(AF2_LOG_CSV)
uid = str(df_af2.iloc[0]["uniprot_id"])
uid_dir = STRUCTURE_HOME / uid

pdb_path = dl_af2_pdb(uid, uid_dir)
assert pdb_path and pdb_path.exists(), f"Could not download AF2 PDB for {uid}"

print(f"Testing AF2 PDB for {uid}: {pdb_path.name} | size(MB)={pdb_path.stat().st_size/1024/1024:.2f}")
parsed = get_struc_seq(str(FOLDSEEK_BIN), str(pdb_path), ["A"], plddt_mask=True)

print("Chains returned:", list(parsed.keys()))
aa, di = parsed["A"][0], parsed["A"][1]
print("AA len, 3Di len:", len(aa), len(di))
print("3Di head:", di[:80])

Testing AF2 PDB for A1K1S0: AF-A1K1S0-F1-model_v4.pdb | size(MB)=0.31
Chains returned: ['A']
AA len, 3Di len: 512 512
3Di head: ###DPVVVVVVVVVCVVPDDP###DWWKWWFQDDDQQKTKTFTPPPDDQQFWKAWPPRWIWGFHDDDPGITIIGTQFDDP


In [ ]:
# ===== Cell 4: Assemble SFT JSONL (prompt, response, aa_seq, stru_str) =====
import json, math
from pathlib import Path
import pandas as pd

WORK_CSV       = OUT_DIR / "worklist_5k.csv"                  # from Cell 1
PER_CHAIN_CSV  = OUT_DIR / "foldseek_3di_per_chain.csv"       # from Cell 3
JSONL_OUT      = OUT_DIR / "sft_protein2desc_seq+3di.jsonl"   # final SFT
CHAIN_POLICY   = "best"    # choose: "best" | "concat" | "all_items"

PROMPT_TEXT = (
    "You are a professional protein biologist. Based on the amino-acid sequence (and structure if available), "
    "write a concise, biologically accurate 2–4 sentence description of the protein."
)

# Load sources
df_work = pd.read_csv(WORK_CSV)              # uniprot_id, seq, response
df_3di  = pd.read_csv(PER_CHAIN_CSV) if PER_CHAIN_CSV.exists() else pd.DataFrame(columns=["uniprot_id","pdb_id","chain","stru_str"])

# Build per-UniProt aggregation of 3Di
agg = {}
for uid, g in df_3di.groupby("uniprot_id"):
    chains = []
    for _, r in g.iterrows():
        chains.append({"pdb_id": str(r["pdb_id"]), "chain": str(r["chain"]), "stru_str": str(r["stru_str"])})
    agg[uid] = chains

# Assemble JSONL
written = 0
with open(JSONL_OUT, "w", encoding="utf-8") as fout:
    for _, row in df_work.iterrows():
        uid  = str(row["uniprot_id"])
        seq  = str(row["seq"])
        resp = str(row["response"])
        chains = agg.get(uid, [])

        if CHAIN_POLICY == "best":
            # choose longest stru_str (proxy for coverage)
            stru_str = ""
            if chains:
                best = max(chains, key=lambda x: len(x["stru_str"]))
                stru_str = best["stru_str"]
            rec = {
                "prompt": PROMPT_TEXT,
                "response": resp,
                "aa_seq": seq
            }
            if stru_str:
                rec["stru_str"] = stru_str
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
            written += 1

        elif CHAIN_POLICY == "concat":
            # concatenate all chain 3Di with single space separator
            stru_str = ""
            if chains:
                stru_str = " ".join([c["stru_str"] for c in chains if c["stru_str"]])
            rec = {
                "prompt": PROMPT_TEXT,
                "response": resp,
                "aa_seq": seq
            }
            if stru_str:
                rec["stru_str"] = stru_str
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
            written += 1

        elif CHAIN_POLICY == "all_items":
            # one JSONL line per (UniProt, chain)
            if not chains:
                # still write a sequence-only example
                rec = {
                    "prompt": PROMPT_TEXT,
                    "response": resp,
                    "aa_seq": seq
                }
                fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
                written += 1
            else:
                for c in chains:
                    if not c["stru_str"]:
                        continue
                    rec = {
                        "prompt": PROMPT_TEXT,
                        "response": resp,
                        "aa_seq": seq,
                        "stru_str": c["stru_str"]
                    }
                    fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    written += 1
        else:
            raise ValueError(f"Unknown CHAIN_POLICY={CHAIN_POLICY}")

print(f"JSONL saved: {JSONL_OUT}  |  lines written: {written}")

# Optional: for 'concat', also save a sidecar listing per-protein chain coverage
if CHAIN_POLICY == "concat":
    side = []
    for uid, chains in agg.items():
        side.append({
            "uniprot_id": uid,
            "n_chains": len(chains),
            "pdb_chain_list": ";".join([f"{c['pdb_id']}:{c['chain']}({len(c['stru_str'])})" for c in chains])
        })
    side_csv = OUT_DIR / "concat_chain_boundaries.csv"
    pd.DataFrame(side).to_csv(side_csv, index=False)
    print("Saved concat chain boundaries to:", side_csv)